In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/*.py
!ls /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/*.py
!ls /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/utils/*.py

'''
!cat '/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/NQTM.py'
!cat '/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/NQTM_v2.py'
!cat '/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/run_NQTM.py'
!cat '/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/run_NQTM_v2.py'
!cat '/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/utils/preprocess.py'
!cat '/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/utils/TU.py'
'''

import sys
sys.path.append('/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere')

%cd /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/

ls: cannot access '/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/*.py': No such file or directory
/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/NQTM.py
/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/NQTM_v2.py
/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/preprocess.py
/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/run_NQTM.py
/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/run_NQTM_v2.py
/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/utils/setup.py
/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/utils/TU.py
/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master


1.   NQTTM





In [ ]:
%pwd

'/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master'

In [ ]:
#!python /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/utils/setup.py build_ext --inplace

In [ ]:
!pip install scikit-learn==1.0.0

In [ ]:
%load_ext cython

In [ ]:
#from Cython import t
%%cython

import os
import cython
import numpy as np
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer
import argparse
import string
from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
import json
import re
import pandas as pd
import time

nltk.download('stopwords')
nltk.download('punkt')
string.punctuation

"""
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', default="/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/data/reddit/Incels.csv")
parser.add_argument('--output_dir', default="/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/input")
parser.add_argument('--name', default='Incels')
parser.add_argument('--min_df', type=int, default=20)
parser.add_argument('--chunksize', type=int, default=50)
args = parser.parse_args()
"""
class Args:
  data_path = "/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/data/reddit/Incels.csv"
  output_dir = "/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/input"
  name = 'Incels'
  min_df = 20
  chunksize = 50
  num_docs = 10000

args=Args()

cdef list texts = []
cdef list stopwords_list = stopwords.words('english')

cdef tuple process(text):
    # remove websites
    try:
        input_str = re.sub(r'http\S+', '', text)

    except TypeError:
        #print(text)
        return None, None


    # lower caps
    input_str = input_str.lower()

    # remove punctuation
    input_str = input_str.translate(
        str.maketrans("", "", string.punctuation)
    )

    # remove trailing spaces
    input_str = input_str.strip()

    # tokenize
    input_str = wordpunct_tokenize(input_str)

    # remove stop words
    cdef list input_str_w_sw = [word for word in input_str if not word in stopwords_list]
    cdef int length = len(input_str_w_sw)
    # filter out short texts
    if length < args.min_df:
        pass
    else:
        input_str = " ".join(input_str_w_sw)
        texts.append(input_str)
    return (length, input_str)

cdef void run():
    """
    with pd.read_csv(args.data_path, chunksize=args.chunksize) as reader:
        for chunk in tqdm(reader, total=1048576/50):
            chunk.apply(lambda x: process(x['text_post']), axis=1, raw=False)
    """
    print('Sarting apply')

    df = pd.read_csv(args.data_path)
    df[['text_length', 'processed_text']] = df.progress_apply(lambda x: process(x['text_post']), axis=1, raw=False, result_type='expand')
    cdef list texts = list(df.nlargest(args.num_docs, 'text_length')['processed_text'])

    with open(f"/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/input/Incels/{args.name}_processed.csv", 'w') as outfile:
        df.to_csv(outfile, index=False)

    with open(f"/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/input/Incels/text_top_{args.num_docs}.txt", 'w') as outfile:
        for t in texts:
            outfile.write('\n'+t)

    print('vectorizing')

    vectorizer = CountVectorizer(min_df=args.min_df)
    bow_matrix = vectorizer.fit_transform(texts).toarray()

    idx = np.where(bow_matrix.sum(axis=-1) > 0)
    bow_matrix = bow_matrix[idx]

    try:
        vocab = vectorizer.get_feature_names_out()
    except AttributeError:
        vocab = vectorizer.get_feature_names()
    print(vocab)

    print("===>saving files")

    os.makedirs(os.path.join(args.output_dir, f'{args.name}'), exist_ok=True)

    scipy.sparse.save_npz(os.path.join(args.output_dir, f'{args.name}', f'bow_matrix.npz'),
                          scipy.sparse.csr_matrix(bow_matrix))
    with open(os.path.join(args.output_dir, f'{args.name}', f'vocab.txt'), 'w') as file:
        for line in vocab:
            file.write(line + '\n')

    print('===>done.')

run()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Sarting apply


100%|██████████| 1191797/1191797 [02:07<00:00, 9343.85it/s] 


vectorizing
['01' '05' '10' ... 'zero' 'zone' 'zoo']
===>saving files
===>done.


In [ ]:
#!python utils/preprocess.py --data_path=/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/data/depression.csv --output_dir=/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/input/reddit --name=depression --chunksize=100


In [ ]:
!python run_NQTM_v2.py --epoch 10 --data_dir /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/input/Incels --output_dir /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/output

In [ ]:
!pip install octis

In [ ]:
import numpy as np
import scipy
import os
from octis.models.model import AbstractModel
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence

def load_data(data_dir:str):
        #print('load_data')
        train_data = scipy.sparse.load_npz(os.path.join(data_dir, 'bow_matrix.npz')).toarray()
        vocab = list()
        with open(os.path.join(data_dir, 'vocab.txt')) as file:
            for line in file:
                vocab.append(line.strip())
        return train_data, vocab

# evaluate
#top_words_T15_K50_1th = np.load('/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/output/top_words_T15_K50_1th')
top_words_T15_K50_1th = []
with open('/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/output/top_words_T15_K50_1th') as file:
    for lines in file:
        l = lines.split()
        top_words_T15_K50_1th.append(l)
beta_K50_1th = np.load('/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/output/beta_K50_1th.npy')
theta_K50_1th = np.load('/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/output/theta_K50_1th.npy')

results = {
    'topics' : np.array(top_words_T15_K50_1th),
    'topic_word_matrix' : beta_K50_1th,
    'topic_doc_matrix' : theta_K50_1th.T
}

train_data, vocab = load_data('/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/input/Incels')

print(results['topics'].shape, beta_K50_1th.shape, theta_K50_1th.T.shape)
print(len(vocab))

(50, 15) (50, 7233) (50, 9999)
7032


In [ ]:
text = []
with open('/content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/input/Incels/text_top_10000.txt') as outfile:
    for l in outfile:
        if l == '\n':
            pass
        else:
            s = l.split()
            text.append(s)

In [ ]:
import matplotlib.pyplot as plt

lengths = [len(t) for t in text]

def condition_0(t):
    return len(t) == 0

def condition_1(t):
    return len(t) > 0

print(sum(map(condition_0, text)))
print(sum(map(condition_1, text)))

0
10000


In [ ]:
topic_diversity = TopicDiversity(topk=5)
coherence = Coherence(texts=text, measure='c_npmi')

scores = [
    topic_diversity.score(results),
    coherence.score(results)
]
print(scores)

In [ ]:
!python /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/utils/TU.py --data_path /content/gdrive/MyDrive/Colab-Notebooks/MLP-Manosphere/NQTM-master/output/top_words_T15_K50_1th

===>TU: 0.600000




> 2. Using the Reddit data



In [ ]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

id2word = corpora.Dictionary(text)
print(type(id2word))
corpus = [id2word.doc2bow(t) for t in text]
print(type(corpus))

cm = CoherenceModel(topics=np.delete(results['topics'], 11, 0),texts = text, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence = cm.get_coherence()

<class 'gensim.corpora.dictionary.Dictionary'>
<class 'list'>


In [ ]:
print(coherence)

-0.03968512937306449


In [ ]:
for w in results['topics'][11]:
    if not(w in id2word.token2id):
        print(w, type(w))

to <class 'numpy.str_'>
the <class 'numpy.str_'>
and <class 'numpy.str_'>
you <class 'numpy.str_'>
of <class 'numpy.str_'>
that <class 'numpy.str_'>
it <class 'numpy.str_'>
in <class 'numpy.str_'>
is <class 'numpy.str_'>
for <class 'numpy.str_'>
are <class 'numpy.str_'>
they <class 'numpy.str_'>
with <class 'numpy.str_'>
have <class 'numpy.str_'>
not <class 'numpy.str_'>


In [ ]:
for i, topics in enumerate(results['topics']):
    ids_from_tokens = [id2word.token2id[t] for t in topics if t in id2word.token2id]
    ids_from_ids = [i for i in topics if i in id2word]

    if len(ids_from_tokens) > len(ids_from_ids):
        pass
    elif len(ids_from_ids) > len(ids_from_tokens):
        pass
    else:
        print(f'{i}')
        print(len(ids_from_tokens))
        print(len(ids_from_ids))
        print(topics)
        print('\n')

11
0
0
['to' 'the' 'and' 'you' 'of' 'that' 'it' 'in' 'is' 'for' 'are' 'they'
 'with' 'have' 'not']


